Подгружаем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('popular')

In [2]:
from nltk.tokenize import word_tokenize

Считвание файла с данными

In [3]:
data = pd.read_csv('test_data.csv')
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


Проверка на пустые значения

In [4]:
data.isnull().sum()

dlg_id    0
line_n    0
role      0
text      0
dtype: int64

Отбор только фраз менеджеров (в задании требуется работа только с ними)

In [5]:
df = data.loc[data['role'] == 'manager', :].reset_index()
df.drop('index', axis=1, inplace=True)
df.head(10)

,dlg_id,line_n,role,text
0,0,1,manager,Алло здравствуйте
1,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
2,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
3,0,8,manager,Угу а на что вы обращаете внимание при выборе
4,0,11,manager,Что для вас приоритет
5,0,15,manager,Ну у вас срок заканчивается поэтому мы набрали...
6,0,29,manager,А так нет не только поэтому просто я обратила ...
7,0,34,manager,А если вы 19 являетесь то лучше то идти бесплатно
8,0,36,manager,Ага хорошо
9,0,45,manager,Индивидуальным поэтому не все то есть сотрудни...


Извлекаем и сохраняем:


1.   Реплики с приветствием – где менеджер поздоровался (greeting_list)
2.   Реплики, где менеджер представил себя (meeting_list)
3.   Имя менеджера (manager_name_list)
4.   Название компании (company_name_list)
5.   Реплики, где менеджер попрощался (goodbye_list)
6.   Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом» (greeting_check_list и goodbye_check_list)







In [6]:
greeting_list = []
meeting_list = []
manager_name_list = []
company_name_list = []
goodbye_list = []
greeting_check_list = []
goodbye_check_list = []

sent = []
for i in range(df.shape[0]):

  #разбиваем строки с текстом на токены
  sent = word_tokenize(df.loc[i, 'text'])
  for j in range(len(sent)):

    #приводим все токены к одному регистру
    sent[j] = sent[j].lower()

    #извлекаем реплики, где менеджер поздоровался с клиентом
    if (sent[j] == "здравствуйте"):
      greeting_list.append(sent)

      #добавляем id диалога в лист для проверки условия из 6 задания
      for counter in range(df.shape[0]):
          if word_tokenize(df['text'][counter].lower()) == sent:
            greeting_check_list.append(df['dlg_id'][counter])

    if (sent[j] == "добрый") & (sent[j] != sent[-1]):
      if (sent[j+1] == "день") | (sent[j+1] == "вечер"):
        greeting_list.append(sent)

        #добавляем id диалога в лист для проверки условия из 6 задания
        for c in range(df.shape[0]):
          if word_tokenize(df['text'][c].lower()) == sent:
            greeting_check_list.append(df['dlg_id'][c])

    #извлекаем реплики, где менеджер представился
    if (sent[j] == "меня") & (sent[j] != sent[-1]):
      if sent[j+1] == "зовут":
        meeting_list.append(sent)
        manager_name_list.append(sent[j+2])

    if (sent[j] == "меня") & (sent[j] != sent[-1]):
      if sent[j+2] == "зовут":
        meeting_list.append(sent)
        manager_name_list.append(sent[j+1])

    #извлекаем имя компании
    if (sent[j] == "компания"):
      company_name_list.append(sent[j+1])

    #извлекаем реплики, где менеджер попрощался с клиентом
    if (sent[j] == "до") & (sent[j] != sent[-1]) | (sent[j] == "всего") & (sent[j] != sent[-1]):
      if (sent[j+1] == "свидания") & (sent[j] != sent[-1]) | (sent[j+1] == "доброго") & (sent[j] != sent[-1]):
        goodbye_list.append(sent)
        
        #добавляем id диалога в лист для проверки условия из 6 задания
        for k in range(df.shape[0]):
          if word_tokenize(df['text'][k].lower()) == sent:
            goodbye_check_list.append(df['dlg_id'][k])


Отбор уникальных значений из листа

In [7]:
def unique(list1):
    x = np.array(list1)
    return list(np.unique(x))

Функция пересечения значений листов. Нужна для 6-го задания (пересечение greeting_check_list и goodbye_check_list)

In [8]:
def intersection(list1, list2):
    res = [value for value in list1 if value in list2]
    return res

Вывод текущих результатов

In [9]:
print(greeting_list)
print(meeting_list)
print(manager_name_list)
print(company_name_list)
print(goodbye_list)
print(unique(greeting_check_list))
print(goodbye_check_list)

[['алло', 'здравствуйте'], ['алло', 'здравствуйте'], ['алло', 'здравствуйте'], ['алло', 'дмитрий', 'добрый', 'день']]
[['меня', 'зовут', 'ангелина', 'компания', 'диджитал', 'бизнес', 'звоним', 'вам', 'по', 'поводу', 'продления', 'лицензии', 'а', 'мы', 'с', 'серым', 'у', 'вас', 'скоро', 'срок', 'заканчивается'], ['меня', 'зовут', 'ангелина', 'компания', 'диджитал', 'бизнес', 'звоню', 'вам', 'по', 'поводу', 'продления', 'а', 'мы', 'сели', 'обратила', 'внимание', 'что', 'у', 'вас', 'срок', 'заканчивается'], ['меня', 'зовут', 'ангелина', 'компания', 'диджитал', 'бизнес', 'звоню', 'вам', 'по', 'поводу', 'продления', 'лицензии', 'а', 'мастера', 'мы', 'с', 'вами', 'сотрудничали', 'по', 'видео', 'там'], ['добрый', 'меня', 'максим', 'зовут', 'компания', 'китобизнес', 'удобно', 'говорить']]
['ангелина', 'ангелина', 'ангелина', 'максим']
['диджитал', 'диджитал', 'диджитал', 'китобизнес', 'хорошо']
[['всего', 'хорошего', 'до', 'свидания'], ['до', 'свидания'], ['угу', 'все', 'хорошо', 'да', 'понеде

Пересечение greeting_check_list и goodbye_check_list

In [10]:
insight = intersection(unique(greeting_check_list), goodbye_check_list)
print(insight)

[0, 1, 3]


In [11]:
grouped_data = data.groupby(['dlg_id']).mean()
grouped_data.drop('line_n', axis=1, inplace=True)

In [12]:
grouped_data

""
dlg_id
0
1
2
3
4
5


In [13]:
grouped_data['insight'] = ''

Конвертирование типов (numpy.array -> int)

In [14]:
for i in range(len(insight)):
  insight[i] = int(insight[i])

In [15]:
for i in range(grouped_data.shape[0]):
  if i in insight:
    grouped_data["insight"][i] = True
  else:
    grouped_data["insight"][i] = False

Проверка требования к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [16]:
grouped_data

,insight
dlg_id,
0,True
1,True
2,False
3,True
4,False
5,False
